# ISP - Interface Segregation Principle

**Nie zmuszaj klientów do implementowania metod, których nie używają.**

Innymi słowy: **Wiele małych interfejsów jest lepsze niż jeden duży.**

## Złe rozwiązanie - łamie ISP

Jeden duży interfejs - "gruby interfejs" (fat interface).

In [1]:
class Machine:
    def print(self, document):
        raise NotImplementedError
    
    def scan(self, document):
        raise NotImplementedError
    
    def fax(self, document):
        raise NotImplementedError

### Przypadek 1: Wielofunkcyjna drukarka - OK

In [4]:
class MultiFunctionPrinter(Machine):
    def print(self, document):
        print(f"Drukuję: {document}")
    
    def scan(self, document):
        print(f"Skanuję: {document}")
    
    def fax(self, document):
        print(f"Wysyłam faks: {document}")


In [5]:
# client code
mfp = MultiFunctionPrinter()
mfp.print("dokument.pdf")
mfp.scan("zdjęcie.jpg")
mfp.fax("raport.doc")

Drukuję: dokument.pdf
Skanuję: zdjęcie.jpg
Wysyłam faks: raport.doc


### Przypadek 2: Zwykła drukarka - problem!

In [6]:
class OldFashionedPrinter(Machine):
    def print(self, document):
        print(f"Drukuję: {document}")  # OK
    
    def scan(self, document):
        pass  # Nie umie skanować, ale musi zaimplementować
    
    def fax(self, document):
        pass  # Nie umie faksować, ale musi zaimplementować

### Problem 1: Puste metody (pass)

```python
printer = OldFashionedPrinter()
printer.scan("zdjęcie.jpg")  # ← Nic się nie dzieje, ale nie ma błędu!
```

**Klient myśli, że scan działa**, ale metoda jest pusta.

---

### Problem 2: Rzucanie wyjątków

In [8]:
class OldFashionedPrinter(Machine):
    def print(self, document):
        print(f"Drukuję: {document}")
    
    def scan(self, document):
        raise NotImplementedError("Printer cannot scan!") 
    
    def fax(self, document):
        raise NotImplementedError("Printer cannot fax!")


In [10]:
# client code
printer = OldFashionedPrinter()
printer.print("dokument.pdf")  # OK

try:
    printer.scan("zdjęcie.jpg")  # Runtime error!
except NotImplementedError as e:
    print(f"{e}")

Drukuję: dokument.pdf
Printer cannot scan!


**Problem:** 
- API pokazuje `scan` i `fax` jako dostępne metody
- Klient myśli, że może ich użyć
- Runtime error psuje działanie aplikacji

**Gruby interfejs psuje spójność swoich podklas.**

## Dobre rozwiązanie - zgodne z ISP

Rozdziel duży interfejs na małe, niezależne interfejsy.

In [11]:
from abc import ABC, abstractmethod

# Małe, wyspecjalizowane interfejsy
class Printer(ABC):
    @abstractmethod
    def print(self, document):
        pass

class Scanner(ABC):
    @abstractmethod
    def scan(self, document):
        pass

class Fax(ABC):
    @abstractmethod
    def fax(self, document):
        pass

### Przypadek 1: Zwykła drukarka - tylko drukowanie

In [13]:
class SimplePrinter(Printer):
    def print(self, document):
        print(f"Drukuję: {document}")


In [14]:
# client code
printer = SimplePrinter()
printer.print("dokument.pdf")  # OK
# printer.scan()  # ← Metoda nie istnieje - brak w API!

Drukuję: dokument.pdf


### Przypadek 2: Kopiarka - drukowanie + skanowanie

In [16]:
class Photocopier(Printer, Scanner):
    def print(self, document):
        print(f"Drukuję: {document}")
    
    def scan(self, document):
        print(f"Skanuję: {document}")


In [17]:
# client code
copier = Photocopier()
copier.print("kopia.pdf")
copier.scan("oryginał.jpg")

Drukuję: kopia.pdf
Skanuję: oryginał.jpg


### Przypadek 3: Wielofunkcyjna drukarka - wszystko

In [18]:
class MultiFunctionDevice(Printer, Scanner, Fax):
    def print(self, document):
        print(f"Drukuję: {document}")
    
    def scan(self, document):
        print(f"Skanuję: {document}")
    
    def fax(self, document):
        print(f"Wysyłam faks: {document}")


In [19]:
# client code
mfd = MultiFunctionDevice()
mfd.print("dokument.pdf")
mfd.scan("zdjęcie.jpg")
mfd.fax("raport.doc")

Drukuję: dokument.pdf
Skanuję: zdjęcie.jpg
Wysyłam faks: raport.doc


## Porównanie

| Aspekt | Złe (gruby interfejs) | Dobre (ISP) |
|--------|-------------------------|----------------|
| Interfejs | 1 duży (print, scan, fax) | 3 małe (Printer, Scanner, Fax) |
| Zwykła drukarka | Musi zaimplementować scan, fax | Implementuje tylko Printer |
| API | Pokazuje metody, których nie ma | Pokazuje tylko dostępne metody |
| Elastyczność | Niska | Wysoka (wybierasz interfejsy) |

## Esencja ISP

### Pytanie:
Czy ta klasa musi implementować metody, których nie używa?

### Złe (łamie ISP):
```python
class Machine:  # Gruby interfejs
    def print(...): pass
    def scan(...): pass
    def fax(...): pass

class SimplePrinter(Machine):
    def print(...): ...  # Używa
    def scan(...): pass  # Nie używa, ale musi zaimplementować
    def fax(...): pass   # Nie używa, ale musi zaimplementować
```

### ✅ Dobre (zgodne z ISP):
```python
class Printer:  # Chudy interfejs
    def print(...): pass

class Scanner:  # Chudy interfejs
    def scan(...): pass

class SimplePrinter(Printer):  # Tylko co potrzebuje
    def print(...): ...  # Implementuje tylko to, co używa
```

---

### Kluczowa reguła:

**Interfejsy powinny być chude, implementacje mogą być grube.**

```python
# Chude interfejsy
class Printer: ...     # 1 metoda
class Scanner: ...     # 1 metoda

# Gruba implementacja
class MultiFunctionDevice(Printer, Scanner, Fax):  # 3 metody
    ...
```

## Podsumowanie

### ISP w jednym zdaniu:
**Wiele małych interfejsów jest lepsze niż jeden duży.**

### Jak rozpoznać łamanie ISP?
- Klasa implementuje metody, które są puste (`pass`)
- Klasa rzuca `NotImplementedError` w niektórych metodach
- Klient widzi metody w API, których nie może użyć

### Jak naprawić?
1. Rozdziel duży interfejs na małe, wyspecjalizowane
2. Klasy implementują tylko potrzebne interfejsy
3. Używaj dziedziczenia wielokrotnego (multiple inheritance)

### Korzyści:
- Czyste API (tylko dostępne metody)
- Brak pustych implementacji
- Łatwiejsze testowanie (mniejsze interfejsy)
- Wyższa spójność klas

### Związek z innymi zasadami:
- **ISP + SRP** - małe interfejsy = pojedyncza odpowiedzialność
- **ISP + DIP** - zależności od małych, wyspecjalizowanych interfejsów